In [ ]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedShuffleSplit
from batchNormalization import BatchNormalization
from affine import Affine
from relu import Relu
from sigmoidWithLoss import SigmoidWithLoss
from model import Model
from optimizer import SGD, Momentum, AdaGrad, RMSprop, Adam
from trainer import Trainer
from util import scale
from metrics import Factory

In [ ]:
#データセット読み込み
data = load_breast_cancer()
x = data.data
y = data.target
ss = StratifiedShuffleSplit(n_splits=2, train_size=0.7, random_state=0)
train_idx, test_idx = ss.split(x, y)
x_train, x_test, t_train, t_test = x[train_idx[0]], x[train_idx[1]], y[train_idx[0]], y[train_idx[1]]

In [ ]:
#学習
m = Model()
m.append(Affine(scale(30) * np.random.randn(30, 100), np.zeros(100)))
m.append(BatchNormalization(np.ones(100), np.zeros(100)))
m.append(Relu())
m.append(Affine(scale(100) * np.random.randn(100, 1), np.zeros(1)))
m.append_loss(SigmoidWithLoss())
m.summary()

In [ ]:
t = Trainer(m, SGD(lr=0.1), Factory.create(Factory.MTYPE.MTR))
t.fit(x_train, t_train, 50, 50)

In [ ]:
#テスト
met = Factory.create(Factory.MTYPE.MTR)
ret = np.array([])
mtr = np.array([])
for i in range(len(x_test)):
    score = m.predict(x_test[[i]])
    ret = np.append(ret, met.evaluate(t_test[[i]], score))
    mtr = np.append(mtr, met.get_matrix(t_test[[i]], score).flatten())
ret = np.sum(ret.reshape(-1, 4), axis=0) / len(x_test)
print('acc %.4f | f1 %.4f' %(ret[0], ret[3]))
np.sum(mtr.reshape(-1, 4), axis=0)